# YOLOv5モデルのファインチューニング

このNotebookでは、COCO事前学習済みYOLOv5nモデルをKaggle OCRデータセットでファインチューニングします。

**高速化のため、ローカルストレージ（/content/tmp/）を使用します。**

## 処理の流れ
1. Google Driveのマウント
2. Gitリポジトリのクローン/更新
3. 環境変数の設定
4. データセットをローカルストレージにコピー（高速化）
5. データセットの確認
6. 事前学習済みモデルの準備
7. 学習設定
8. 学習の実行
9. 学習結果の確認
10. 最良モデルをGoogle Driveに保存


## 使用方法について

このNotebookは`create_yolo_dataset.ipynb`の後に実行してください。

- **前提**: `create_yolo_dataset.ipynb`で画像とラベルの1対1対応が完成していること
- **入力**: `processed/{split}/images/` と `processed/{split}/labels/`
- **出力**: 学習済みモデル（`.pt`ファイル）をGoogle Driveに保存
- **結果**: ファインチューニング済みYOLOv5nモデル

### 注意事項
- **ローカルストレージを使用**: Google Driveへの直接アクセスは遅いため、一時的にローカルストレージ（/content/tmp/）を使用します
- **GPU使用**: ColabのGPUランタイムを使用してください（ランタイム → ランタイムのタイプを変更 → GPU）
- **段階的fine-tuning**: このNotebookは段階2（Kaggleデータ）用です。段階3（自前データ）は別途実行してください


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%cd /content
!git clone -b feature/onnx_recognizer https://github.com/masararutaru/last_assignment_progzissen.git
# %cd last_assignment_progzissen
# !git pull  # 毎回実行

# 環境変数の設定
import os
os.environ["DATA_ROOT"] = "/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment"
os.environ["OUT_ROOT"] = "/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment"

print("環境変数設定完了")
print(f"DATA_ROOT: {os.environ.get('DATA_ROOT')}")
print(f"OUT_ROOT: {os.environ.get('OUT_ROOT')}")


## データセットをローカルストレージにコピー（高速化）

Google Driveから直接読み込むと遅いため、ローカルストレージにコピーします。
**初回のみ実行が必要です（数分かかります）。**


In [ ]:
# データセットをローカルストレージにコピー
import shutil
from pathlib import Path
from tqdm import tqdm

# 環境変数からパスを取得
out_root = os.environ.get("OUT_ROOT")
if not out_root:
    raise ValueError(
        "❌ エラー: OUT_ROOT環境変数が設定されていません。\n"
        "   セル3で環境変数を設定してください。"
    )

split = 'train'  # 'train', 'val', 'test' を変更可能

# Google Driveのデータセットパス
drive_data_dir = Path(out_root) / "processed" / split
drive_images_dir = drive_data_dir / "images"
drive_labels_dir = drive_data_dir / "labels"

# ローカルストレージのパス
local_data_dir = Path("/content/tmp/dataset") / split
local_images_dir = local_data_dir / "images"
local_labels_dir = local_data_dir / "labels"

print("=" * 80)
print("データセットのローカルコピー")
print("=" * 80)
print(f"ソース（Google Drive）: {drive_data_dir}")
print(f"  画像: {drive_images_dir}")
print(f"  ラベル: {drive_labels_dir}")
print(f"出力先（ローカル）: {local_data_dir}")
print()

# 既にコピー済みか確認
if local_images_dir.exists() and local_labels_dir.exists():
    image_count = len(list(local_images_dir.glob('*.*')))
    label_count = len(list(local_labels_dir.glob('*.txt')))
    print(f"✓ ローカルデータセットは既に存在します")
    print(f"  画像ファイル数: {image_count} 件")
    print(f"  ラベルファイル数: {label_count} 件")
    print(f"\n再コピーする場合は、以下のコマンドを実行してください:")
    print(f"  !rm -rf {local_data_dir}")
else:
    # ソースの存在確認
    if not drive_images_dir.exists():
        raise FileNotFoundError(
            f"❌ エラー: 画像ディレクトリが見つかりません: {drive_images_dir}\n"
            "   先に create_yolo_dataset.ipynb を実行してください。"
        )
    
    if not drive_labels_dir.exists():
        raise FileNotFoundError(
            f"❌ エラー: ラベルディレクトリが見つかりません: {drive_labels_dir}\n"
            "   先に preprocess_kaggle_data.ipynb を実行してください。"
        )
    
    # ファイルリストを取得
    print("ファイルリストを取得中...")
    image_files = list(drive_images_dir.glob('*.*'))
    label_files = list(drive_labels_dir.glob('*.txt'))
    
    print(f"\n見つかったファイル:")
    print(f"  画像ファイル: {len(image_files)} 件")
    print(f"  ラベルファイル: {len(label_files)} 件")
    print()
    
    # ディレクトリ作成
    local_images_dir.mkdir(parents=True, exist_ok=True)
    local_labels_dir.mkdir(parents=True, exist_ok=True)
    
    # 画像ファイルをコピー（進捗表示付き）
    print("=" * 80)
    print("画像ファイルをコピー中...")
    print("=" * 80)
    copied_images = 0
    skipped_images = 0
    
    for img_file in tqdm(image_files, desc="画像コピー", unit="ファイル"):
        target_path = local_images_dir / img_file.name
        if target_path.exists():
            skipped_images += 1
            continue
        try:
            shutil.copy2(img_file, target_path)
            copied_images += 1
        except Exception as e:
            print(f"\n⚠️  エラー: {img_file.name} のコピーに失敗: {e}")
    
    print(f"\n画像コピー完了: コピー {copied_images} 件, スキップ {skipped_images} 件")
    
    # ラベルファイルをコピー（進捗表示付き）
    print("\n" + "=" * 80)
    print("ラベルファイルをコピー中...")
    print("=" * 80)
    copied_labels = 0
    skipped_labels = 0
    
    for label_file in tqdm(label_files, desc="ラベルコピー", unit="ファイル"):
        target_path = local_labels_dir / label_file.name
        if target_path.exists():
            skipped_labels += 1
            continue
        try:
            shutil.copy2(label_file, target_path)
            copied_labels += 1
        except Exception as e:
            print(f"\n⚠️  エラー: {label_file.name} のコピーに失敗: {e}")
    
    print(f"\nラベルコピー完了: コピー {copied_labels} 件, スキップ {skipped_labels} 件")
    
    print("\n" + "=" * 80)
    print("✓ コピー完了")
    print("=" * 80)
    print(f"画像ファイル数: {len(image_files)} 件")
    print(f"ラベルファイル数: {len(label_files)} 件")
    print(f"\nローカルパス: {local_data_dir}")


In [ ]:
# データセットの確認
import sys
from pathlib import Path
from collections import defaultdict

# パスを追加（リポジトリのpython/dataディレクトリ）
repo_path = Path('/content/last_assignment_progzissen')
sys.path.insert(0, str(repo_path / 'python' / 'data'))

from config import CLASSES

# ローカルデータセットのパス
local_data_dir = Path("/content/tmp/dataset") / split
local_images_dir = local_data_dir / "images"
local_labels_dir = local_data_dir / "labels"

print("=" * 80)
print("データセット確認")
print("=" * 80)

# ファイル数の確認
print("\nファイル数を確認中...")
image_files = list(local_images_dir.glob('*.*'))
label_files = list(local_labels_dir.glob('*.txt'))

print(f"\n画像ファイル数: {len(image_files)} 件")
print(f"ラベルファイル数: {len(label_files)} 件")

# 1対1対応の確認
image_uuids = {f.stem for f in image_files}
label_uuids = {f.stem for f in label_files}
matched = image_uuids & label_uuids

print(f"\n1対1対応:")
print(f"  画像ファイル: {len(image_uuids)} 件")
print(f"  ラベルファイル: {len(label_uuids)} 件")
print(f"  対応しているペア数: {len(matched)} 件")

if len(matched) != len(image_uuids) or len(matched) != len(label_uuids):
    print(f"\n⚠️  警告: 一部のファイルが対応していません")
    if len(image_uuids) != len(label_uuids):
        print(f"  画像のみ: {len(image_uuids - label_uuids)} 件")
        print(f"  ラベルのみ: {len(label_uuids - image_uuids)} 件")

# クラス別統計（進捗表示付き）
print("\n" + "=" * 80)
print("クラス別統計を集計中...")
print("=" * 80)

class_stats = defaultdict(int)
total_instances = 0

for label_file in tqdm(label_files, desc="ラベル解析", unit="ファイル"):
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                if 0 <= class_id < len(CLASSES):
                    class_stats[CLASSES[class_id]] += 1
                    total_instances += 1

print(f"\nクラス別インスタンス数:")
print("-" * 60)
for cls in CLASSES:
    count = class_stats[cls]
    print(f"  {cls:>3}: {count:>8} インスタンス")
print("-" * 60)
print(f"総インスタンス数: {total_instances}")

if total_instances == 0:
    print("\n⚠️  警告: インスタンスが見つかりませんでした")
    print("   データセットを確認してください。")


In [ ]:
# ultralyticsのインストール（必要に応じて）
!pip install ultralytics -q

from ultralytics import YOLO
from pathlib import Path

print("=" * 80)
print("事前学習済みモデルの準備")
print("=" * 80)

# YOLOv5nモデルを読み込み（初回実行時に自動ダウンロード）
print("\nYOLOv5nモデルを読み込み中...")
print("（初回実行時は自動ダウンロードされます。約4.5MB、数秒かかります）")

model = YOLO('yolov5n.pt')

print("\n✓ モデル読み込み完了")
print(f"  モデル名: YOLOv5n")
print(f"  クラス数: {len(CLASSES)} クラス")
print(f"  クラス: {', '.join(CLASSES)}")


## 学習設定

学習パラメータを設定します。段階2（Kaggleデータ）用の設定です。


In [ ]:
# 学習設定
from pathlib import Path

# データセットパス（ローカルストレージ）
data_path = str(Path("/content/tmp/dataset") / split)

# 学習パラメータ
epochs = 100  # エポック数
batch_size = 16  # バッチサイズ（GPUメモリに応じて調整）
img_size = 640  # 画像サイズ
device = 0  # GPU使用（0: GPU, cpu: CPU）

# 出力先（ローカルストレージ）
project_path = "/content/tmp/runs"
run_name = f"yolov5n_kaggle_{split}"

print("=" * 80)
print("学習設定")
print("=" * 80)
print(f"データセット: {data_path}")
print(f"エポック数: {epochs}")
print(f"バッチサイズ: {batch_size}")
print(f"画像サイズ: {img_size}")
print(f"デバイス: {'GPU' if device == 0 else 'CPU'}")
print(f"出力先: {project_path}/{run_name}")
print(f"\nクラス数: {len(CLASSES)} クラス")
print(f"クラス: {', '.join(CLASSES)}")


## 学習の実行

**注意**: このセルは長時間かかります（数時間）。GPUランタイムを使用してください。


In [ ]:
# 学習の実行
print("=" * 80)
print("学習開始")
print("=" * 80)
print(f"開始時刻: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# 学習実行
results = model.train(
    data=data_path,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    device=device,
    project=project_path,
    name=run_name,
    save=True,
    save_period=10,  # 10エポックごとに保存
    val=True,  # 検証データで評価
    plots=True,  # 学習曲線をプロット
    verbose=True,  # 詳細ログ
)

print("\n" + "=" * 80)
print("学習完了")
print("=" * 80)
print(f"終了時刻: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


## 学習結果の確認


In [ ]:
# 学習結果の確認
from pathlib import Path
import json

results_dir = Path(project_path) / run_name
weights_dir = results_dir / "weights"

print("=" * 80)
print("学習結果")
print("=" * 80)
print(f"結果ディレクトリ: {results_dir}")

# 重みファイルの確認
if weights_dir.exists():
    weight_files = list(weights_dir.glob('*.pt'))
    print(f"\n重みファイル数: {len(weight_files)} 件")
    
    for wf in sorted(weight_files):
        size_mb = wf.stat().st_size / (1024 * 1024)
        print(f"  {wf.name}: {size_mb:.2f} MB")
    
    # 最良モデル
    best_model = weights_dir / "best.pt"
    if best_model.exists():
        print(f"\n✓ 最良モデル: {best_model.name}")
        print(f"  サイズ: {best_model.stat().st_size / (1024 * 1024):.2f} MB")
    
    # 最終モデル
    last_model = weights_dir / "last.pt"
    if last_model.exists():
        print(f"✓ 最終モデル: {last_model.name}")
        print(f"  サイズ: {last_model.stat().st_size / (1024 * 1024):.2f} MB")
else:
    print("\n⚠️  重みファイルが見つかりません")

# 学習曲線の確認
results_csv = results_dir / "results.csv"
if results_csv.exists():
    print(f"\n学習曲線データ: {results_csv}")
    print("  （results.csvをダウンロードしてExcel等で確認できます）")

# プロット画像の確認
plot_files = list(results_dir.glob("*.png"))
if len(plot_files) > 0:
    print(f"\nプロット画像数: {len(plot_files)} 件")
    for pf in plot_files:
        print(f"  {pf.name}")


## 最良モデルをGoogle Driveに保存

学習済みモデルをGoogle Driveに保存して、次回以降も使用できるようにします。


In [ ]:
# 最良モデルをGoogle Driveに保存
import shutil
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# 環境変数からパスを取得
out_root = os.environ.get("OUT_ROOT")
if not out_root:
    raise ValueError(
        "❌ エラー: OUT_ROOT環境変数が設定されていません。\n"
        "   セル3で環境変数を設定してください。"
    )

# ローカルの最良モデル
local_best_model = Path(project_path) / run_name / "weights" / "best.pt"

if not local_best_model.exists():
    raise FileNotFoundError(
        f"❌ エラー: 最良モデルが見つかりません: {local_best_model}\n"
        "   先に学習を実行してください。"
    )

# Google Driveの保存先
drive_weights_dir = Path(out_root) / "weights"
drive_weights_dir.mkdir(parents=True, exist_ok=True)

# ファイル名（タイムスタンプ付き）
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
drive_model_name = f"yolov5n_kaggle_{split}_{timestamp}.pt"
drive_best_model = drive_weights_dir / drive_model_name

print("=" * 80)
print("最良モデルをGoogle Driveに保存")
print("=" * 80)
print(f"ソース: {local_best_model}")
print(f"保存先: {drive_best_model}")

# ファイルサイズを取得して進捗表示
file_size = local_best_model.stat().st_size
size_mb = file_size / (1024 * 1024)
print(f"\nファイルサイズ: {size_mb:.2f} MB")
print("コピー中...")

# コピー（大きなファイルの場合は進捗表示）
if file_size > 10 * 1024 * 1024:  # 10MB以上の場合
    # チャンクごとにコピーして進捗表示
    chunk_size = 1024 * 1024  # 1MB
    with open(local_best_model, 'rb') as src, open(drive_best_model, 'wb') as dst:
        with tqdm(total=file_size, unit='B', unit_scale=True, desc="コピー中") as pbar:
            while True:
                chunk = src.read(chunk_size)
                if not chunk:
                    break
                dst.write(chunk)
                pbar.update(len(chunk))
else:
    # 小さいファイルは通常のコピー
    shutil.copy2(local_best_model, drive_best_model)

print(f"\n✓ 保存完了")
print(f"  ファイル名: {drive_model_name}")
print(f"  サイズ: {size_mb:.2f} MB")
print(f"\n次回以降は以下のパスから読み込めます:")
print(f"  {drive_best_model}")
